In [1]:
# Import Libraries
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import collections
import operator

from datetime import datetime
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn import svm 
from sklearn import tree
from sklearn import neural_network
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier



# import random
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import KFold

In [2]:
### Helper Functions

def Accuracy(Y_true, Y_prob):
    """
    Prediction accuracy
    """
    #return accuracy_score(Y_true, np.argmax(Y_prob, axis=1))
    return accuracy_score(Y_true, Y_prob)

def Preprocess(X):
    print("preprocessing...")
    # Mean subtraction to centre the data
    X_mean = np.mean(X, axis = 0)
    X = X - X_mean
    # Standardization
    X /= np.std(X, axis = 0)
    return X

def Predict(model, X):
    p = model.predict(X)
    return p

def Visualize(X, Y):
    plt.scatter(X[:,0], X[:,1],c=Y)
    plt.show()
       

In [16]:
### Load Dataset

# Full Data
proj = pd.read_csv('ks-projects-201801.csv')
x = proj.values
# Sample of data
xsdf = pd.read_csv('sampledata.csv')
xs = xsdf.values

In [19]:
# Slicing just successful and failed projects
proj = proj[(proj['state'] == 'failed') | (proj['state'] == 'successful')]

# Remove outlier usd_goal_real
outlier_cost = 10000
proj = proj[proj['usd_goal_real'] < outlier_cost]

proj.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
6,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21,1000.0,2014-12-01 18:30:44,1205.0,successful,16,US,1205.0,1205.0,1000.00
10,100004721,Of Jesus and Madmen,Nonfiction,Publishing,CAD,2013-10-09,2500.0,2013-09-09 18:19:37,0.0,failed,0,CA,0.0,0.0,2406.39
12,1000055792,The Cottage Market,Crafts,Crafts,USD,2014-10-02,5000.0,2014-09-02 17:11:50,0.0,failed,0,US,0.0,0.0,5000.00


In [ ]:
### Determine the time duration

for i in proj.index:
    deadline = datetime.strptime(proj['deadline'][i], '%Y-%m-%d')
    launched = datetime.strptime(proj['launched'][i], '%Y-%m-%d %X')
    proj['deadline'][i] = (deadline - launched).total_seconds()/3600/24


/Users/mvanheukelom/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
data = proj[['state', 'main_category', 'country', 'usd_goal_real', 'deadline']]
data.head()

In [ ]:
# Modifing value of dependent variable from categorical to numerical
data.loc[data['state'] == 'failed', 'state'] = 0
data.loc[data['state'] == 'successful', 'state'] = 1

# Scaling goal amount since it behaves differently in each category
for cat in data['main_category'].unique():
    scaler = StandardScaler()
    new_values = scaler.fit_transform(data[data['main_category'] == cat][['usd_goal_real']])
    data.loc[proj['main_category'] == cat, 'usd_goal_real'] = new_values.transpose()[0]
    
data.head()

In [34]:
data = pd.get_dummies(data)
print(data.shape)

(331675, 40)


<class 'pandas.core.frame.DataFrame'>
